In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
%matplotlib inline

In [2]:
pip install pyforest

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyforest import *
lazy_imports()

['from sklearn.linear_model import LinearRegression',
 'import numpy as np',
 'import sys',
 'from pathlib import Path',
 'import os',
 'import plotly as py',
 'from sklearn.linear_model import Lasso',
 'from openpyxl import load_workbook',
 'import fbprophet',
 'from sklearn.model_selection import train_test_split',
 'import statsmodels.api as sm',
 'from sklearn.linear_model import ElasticNetCV',
 'from sklearn.feature_extraction.text import CountVectorizer',
 'import nltk',
 'from scipy import stats',
 'from sklearn.impute import SimpleImputer',
 'from statsmodels.tsa.arima_model import ARIMA',
 'from sklearn.linear_model import ElasticNet',
 'from sklearn.preprocessing import StandardScaler',
 'from sklearn.manifold import TSNE',
 'import pandas as pd',
 'import fastai',
 'from sklearn.cluster import KMeans',
 'from sklearn.linear_model import LassoCV',
 'import matplotlib as mpl',
 'from fbprophet import Prophet',
 'from sklearn.model_selection import RandomizedSearchCV',
 'import

In [4]:
df = pd.read_csv('data-Slump.csv')

<IPython.core.display.Javascript object>

In [5]:
df.columns = ['fineagg','coarseagg','ash','Silica_Fume','cement', 'slag','Water', 'superplastic', 'slump']

In [6]:
df.head()

,fineagg,coarseagg,ash,Silica_Fume,cement,slag,Water,superplastic,slump
0,677.0,1230.0,0.0,0.0,440.0,0.0,167.0,8.8,11.0
1,621.0,1232.0,132.0,0.0,308.0,0.0,167.0,8.8,12.3
2,613.0,1270.0,88.0,22.0,308.0,22.0,167.0,8.8,13.6
3,613.0,1270.0,66.0,33.0,308.0,33.0,167.0,8.8,13.5
4,647.0,1177.0,220.0,0.0,220.0,0.0,167.0,8.8,14.0


In [7]:
for cols in df.columns[:-1]:
  Q1 = df[cols].quantile(0.25)
  Q3 = df[cols].quantile(0.75)
  iqr = Q3 - Q1

  low = Q1-1.5*iqr
  high = Q3+1.5*iqr
  df.loc[(df[cols] < low) | (df[cols] > high), cols] = df[cols].median()

In [8]:
#Splitting the data into independent and dependent attributes


#independent and dependent variables
X = df.drop('slump', axis = 1)
y = df['slump']

In [9]:
from scipy.stats import zscore

Xscaled = X.apply(zscore)
Xscaled_df = pd.DataFrame(Xscaled, columns=df.columns)

<IPython.core.display.Javascript object>

In [10]:
X_train, X_test, y_train, y_test = train_test_split(Xscaled,y, test_size= 0.15, random_state=1 )

<IPython.core.display.Javascript object>

In [11]:
from sklearn.metrics import mean_squared_error, r2_score

In [12]:
Rmodel = GradientBoostingRegressor()
Rmodel.fit(X_train, y_train)

<IPython.core.display.Javascript object>

GradientBoostingRegressor()

In [13]:
# Predict and evaluate the model
y_pred = Rmodel.predict(X_test)
initial_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
initial_r2 = r2_score(y_test, y_pred)

<IPython.core.display.Javascript object>

In [14]:
# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 20],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 2]
}
grid_search = GridSearchCV(estimator=Rmodel, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

<IPython.core.display.Javascript object>

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END 

GridSearchCV(cv=3, estimator=GradientBoostingRegressor(),
             param_grid={'max_depth': [None, 20], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 10],
                         'n_estimators': [100, 200]},
             scoring='neg_mean_squared_error', verbose=2)

In [15]:
# Best model from grid search
best_rf = grid_search.best_estimator_

In [16]:
# Predict with the best model
final_y_pred = best_rf.predict(X_test)
final_rmse = np.sqrt(mean_squared_error(y_test, final_y_pred))
final_r2 = r2_score(y_test, final_y_pred)

<IPython.core.display.Javascript object>

In [17]:
# Predict on new data
# new_data = {
#     'fineagg': [float(input("Fine Aggregate (kg/m^3): "))],
#     'coarseagg': [float(input("Coarse Aggregate (kg/m^3): "))],
#     'ash': [float(input("Fly ash (kg/m^3): "))],
#     'Silica_Fume': [float(input("Silica Fume (kg/m^3): "))],
#     'cement': [float(input("Cement (kg/m^3): "))],
#     'slag': [float(input("Slag (kg/m^3): "))],
#     'Water': [float(input("Water (kg/m^3): "))],
#     'superplastic': [float(input("Superplasticizer (kg/m^3): "))],    
# }
# new_df = pd.DataFrame(new_data)
# new_pred = best_rf.predict(new_df)
# print("Predicted Compressive Strength:", new_pred[0])

In [18]:
#saving best model
import pickle
savePath = r'models/grad_boost.sav'
pickle.dump(best_rf,open(savePath, 'wb'))